In [1]:
# 查看gpu情况
!nvidia-smi

Tue Sep 24 16:24:27 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   35C    P8     3W /  N/A |      0MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn
# cuda:0 和 cuda等价，这里定义了两个显卡的cuda
torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [4]:
# 可用cuda数量查询，也就是gpu数量
torch.cuda.device_count()
# 获取设备，如果没有gpu则返回cpu
def try_gpu(i=0):
    if torch.cuda.device_count()>=i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]
try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [17]:
# 张量 存储在gpu上
X = torch.ones(2,3,device=try_gpu(1))
Y = torch.rand(2,3,device=try_gpu())
print(X,Y)
# 计算X+Y 如果X再cuda：1，需要迁移到同样的cuda上才能计算
Z = X.cuda(0) # 迁移到cuda0
print(Z)
print(Y+Z) # 必须要在一个cuda才能计算
Z.cuda(0) is Z


tensor([[1., 1., 1.],
        [1., 1., 1.]]) tensor([[0.6528, 0.2944, 0.2350],
        [0.8069, 0.1781, 0.2359]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1.6528, 1.2944, 1.2350],
        [1.8069, 1.1781, 1.2359]], device='cuda:0')


True

In [19]:
# 模型 存储在gpu上
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())
X=X.cuda(0)
net(X)

tensor([[0.1600],
        [0.1600]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [22]:
net[0].weight.data.device

device(type='cuda', index=0)